## Libraries

In [ ]:
# client.close()

In [ ]:
import hvplot.pandas
import holoviews as hv, geoviews as gv, datashader as ds, param as pm, panel as pn
from holoviews.operation.datashader import datashade, dynspread
import datashader.spatial as dsp
from datashader.utils import lnglat_to_meters
from holoviews.streams import BoxEdit
from dask.distributed import Client

client = Client()

hv.extension('bokeh', logo=False, width=100)

## Data

In [ ]:
sddf = dsp.read_parquet("data/ais_spatial.parquet")
sddf = sddf.persist()

In [ ]:
spoints = hv.Points(sddf, kdims=["x", "y"])

## Main Dashboard

In [ ]:
class AISExplorer(pm.Parameterized):
    basemap           = pm.Selector(gv.tile_sources.tile_sources, default=gv.tile_sources.Wikipedia)
    basemap_opacity   = pm.Magnitude(0.7)
    data_opacity      = pm.Magnitude(1.0)
    show_locations    = pm.Boolean(False)

    @pm.depends("basemap", "basemap_opacity")
    def get_basemap(self):
        return self.basemap.opts(height=600, width=800,
                                    alpha=self.basemap_opacity)
    
    @pm.depends('show_locations')
    def get_labels(self):
        return gv.tile_sources.StamenLabels.opts(level='annotation', alpha=1 if self.show_locations else 0)
    
    def viewable(self, **kwargs):
        return (hv.DynamicMap(self.get_basemap) * 
                dynspread(datashade(spoints, cmap=ds.colors.viridis)).apply.opts(alpha=self.param.data_opacity) *
                hv.DynamicMap(self.get_labels) * 
                box_poly)
    
ais_explorer = AISExplorer(name="")

## Other plots

In [ ]:
def count_ais_pings_by_mmsi(data):
    min_x, min_y = lnglat_to_meters(data["x0"], data["y0"])
    max_x, max_y = lnglat_to_meters(data["x1"], data["y1"])

    x_range = (min_x[0], max_x[0])
    y_range = (min_y[0], max_y[0])

    sddf_box_rough = sddf.spatial_query(x_range, y_range)
    sddf_box_fine = sddf_box_rough.query(f"x > {x_range[0]} & x < {x_range[1]} & " +
                                         f"y > {y_range[0]} & y < {y_range[1]}")
    
    s_value_counts = sddf_box_fine.mmsi.value_counts().compute()[:10].sort_values()

    return s_value_counts.hvplot.bar(title="Count of AIS pings by MMSI (top 10), within blue box",
                                     hover_alpha=0.5,
                                     ylabel="AIS Ping Count",
                                     xlabel="MMSI",
                                     invert=True).opts(width=1000)

In [ ]:
sample_box = hv.Bounds((0, 0, 1, 1))
box_poly = gv.Polygons([sample_box])
box_stream = BoxEdit(source=box_poly, num_objects=1)

## Serve

In [ ]:
panel = pn.Column(
    pn.Row(pn.Column(pn.Pane("#AIS Explorer"),
                     pn.Param(ais_explorer.param, expand_button=False)),
           ais_explorer.viewable()),
    pn.Row(hv.DynamicMap(count_ais_pings_by_mmsi, streams=[box_stream]))
)

panel.servable()

In [ ]:
# panel.save("app.png")